### タスク 1.1: 基盤セットアップと基本的なエージェントの作成

Strands Agents フレームワークを使用してカスタマーサポートエージェントのプロトタイプをセットアップします。このプロトタイプは、エージェントのプロトタイプから本番環境対応のソリューションまでの完全な旅を探索するための出発点となります。

このタスクを完了すると、エージェントは以下の基本的なアーキテクチャを持つようになります:

<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*画像の説明: ローカルツールを使用してローカルで実行されるシンプルなエージェントプロトタイプ*

依存関係をインストールし、AWS SDK、AgentCore コンポーネント、Strands フレームワークを含むすべての必要なライブラリをインポートして、開発環境を準備します。

#### 本来は下記のような pip コマンドで必要なパッケージをインストールします (この環境ではすでにインストール済ですので実行不要です）
%pip install -U -r requirements.txt -q --no-warn-conflicts 

#### 下記のコードセルから順に実行します。

In [ ]:
import boto3
import json
import uuid
import time
import requests
from boto3.session import Session

# AgentCore のインポート
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

# Strands のインポート
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
from mcp.client.streamable_http import streamablehttp_client

# ローカルツール
from lab_helpers.lab1_strands_agent import (
    get_product_info, get_return_policy, get_technical_support, web_search,
    SYSTEM_PROMPT, MODEL_ID
)
from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter
from scripts.utils import get_cognito_client_secret

# セットアップ
boto_session = Session()
REGION = boto_session.region_name
CUSTOMER_ID = "customer_001"
SESSION_ID = str(uuid.uuid4())

print("✅ Libraries imported successfully!")

エージェントを作成する前に、カスタマーサポート機能を強化するローカルツールを確認します。`lab_helpers/lab1_strands_agent.py` を開いて確認してください:

- ツールは `@tool` デコレータを使用してこのファイルでローカルに定義されています
- 4つのツール関数とその目的:
  - get_product_info(): 製品情報を取得
  - get_return_policy(): 特定の製品の返品ポリシーを取得
  - get_technical_support(): 技術サポートガイダンスを提供
  - web_search(): 最新情報をウェブで検索
- モックデータの使用方法(実際のデータベース/APIをシミュレート)
- エージェントの動作を定義するシステムプロンプト

エージェントを作成する前に、カスタマーサポート機能を強化するローカルツールを確認します。`lab_helpers/lab1_strands_agent.py` を開いて確認してください:

- ツールは `@tool` デコレータを使用してこのファイルでローカルに定義されています
- 4つのツール関数とその目的:
  - get_product_info(): 製品情報を取得
  - get_return_policy(): 特定の製品の返品ポリシーを取得
  - get_technical_support(): 技術サポートガイダンスを提供
  - web_search(): 最新情報をウェブで検索
- モックデータの使用方法(実際のデータベース/APIをシミュレート)
- エージェントの動作を定義するシステムプロンプト

In [ ]:
# ローカルツールを使用した基本的なエージェントを作成
model = BedrockModel(model_id=MODEL_ID, temperature=0.3, region_name=REGION)
basic_agent = Agent(
    model=model,
    tools=[get_product_info, get_return_policy, get_technical_support, web_search],
    system_prompt=SYSTEM_PROMPT
)

print("✅ Basic customer support agent ready!")
print("📋 Available tools: Product Info, Return Policy, Technical Support, Web Search")

基本的なカスタマーサポートエージェントを作成します。このエージェントは、クエリの理解からアクションの実行まで、コアAI機能を実証します。このエージェントは以下を組み合わせています:

- **基盤モデル**: 推論と意思決定を支える「脳」
- **システムプロンプト**: エージェントの性格とサービス基準を定義する行動指示
- **専門ツール**: 4つのローカルツール(製品情報、返品ポリシー、技術サポート、ウェブ検索)

エージェントを呼び出すと、以下のプロセスに従います:
1. **クエリ分析**: エージェントが顧客の質問を分析
2. **ツール選択**: エージェントが使用するツールを決定(必要に応じて)
3. **ツール実行**: エージェントが適切なパラメータで適切なツールを呼び出し
4. **応答の合成**: エージェントがツールの結果と知識を組み合わせて有用な応答を作成
5. **品質チェック**: エージェントがシステムプロンプトの基準を満たしているか確認

In [ ]:
# 基本的なエージェントの機能をテスト
print("💬 Testing basic agent...\n")
response = basic_agent("Laptopの返品ポリシーは何ですか??")
print("\n" + "="*50 + "\n")

基本的なエージェントをテストして、顧客のクエリをどのように処理し、ツールを使用するかを確認します:

In [ ]:
# AgentCore Memory サービス用のメモリクライアントを初期化
memory_client = MemoryClient(region_name=REGION)
memory_name = "CustomerSupportMemory"

def create_or_get_memory_resource():
    try:
        # SSM パラメータから既存のメモリリソースを取得を試行
        memory_id = get_ssm_parameter("/app/customersupport/agentcore/memory_id")
        memory_client.gmcp_client.get_memory(memoryId=memory_id)
        return memory_id
    except:
        # 2つの戦略で新しいメモリリソースを作成
        strategies = [
            {
                # USER_PREFERENCE 戦略は顧客の好みと行動をキャプチャ
                StrategyType.USER_PREFERENCE.value: {
                    "name": "CustomerPreferences",
                    "description": "Captures customer preferences and behavior",
                    "namespaces": ["support/customer/{actorId}/preferences"],
                }
            },
            {
                # SEMANTIC 戦略は会話からの事実情報を保存
                StrategyType.SEMANTIC.value: {
                    "name": "CustomerSupportSemantic",
                    "description": "Stores facts from conversations",
                    "namespaces": ["support/customer/{actorId}/semantic"],
                }
            },
        ]
        print("Creating AgentCore Memory resources (2-3 minutes)...")
        # メモリリソースを作成して完了を待機
        response = memory_client.create_memory_and_wait(
            name=memory_name,
            description="Customer support agent memory",
            strategies=strategies,
            event_expiry_days=90,  # メモリイベントは90日後に期限切れ
        )
        memory_id = response["id"]
        # 将来の使用のためにメモリIDをSSMに保存
        put_ssm_parameter("/app/customersupport/agentcore/memory_id", memory_id)
        return memory_id

memory_id = create_or_get_memory_resource()
print(f"✅ Memory resource ready: {memory_id}")

この初期プロトタイプには、後続のタスクで対処するいくつかの制限があります:

- **永続的なメモリがない** - エージェントは以前のセッションからの顧客履歴と好みを忘れる
- **ローカルツールのみ** - 共有または企業グレードのツール統合がない
- **アイデンティティ管理がない** - 特定のユーザーの代わりに行動できない

In [ ]:
# 以前の顧客のやり取りをシード
previous_interactions = [
    ("ビデオ編集中にMacBook Proが過熱する問題が発生しています.", "USER"),
    ("熱の問題についてはお手伝いできます。ご注文番号MB-78432のMacBook Proはまだ保証期間内です。", "ASSISTANT"),
    ("ゲーミングヘッドフォンの返品ポリシーは？FPSの対戦ゲームには低遅延が必要です", "USER"),
    ("ゲーミングヘッドフォンは30日間返品可能です。FPSの競技志向なので、オーディオレイテンシーの仕様を確認することをお勧めします。", "ASSISTANT"),
    ("プログラミング用に1200ドル以下のノートパソコンを探しています。最低16GBのRAMがあり、Linuxとの互換性が良いものが望ましいです。ThinkPadモデルが好みです。", "USER"),
    ("完璧です！開発作業には、Linux サポートが優れた ThinkPad E シリーズまたは Dell XPS モデルをお勧めします。", "ASSISTANT"),
]

if memory_id:
    memory_client.create_event(
        memory_id=memory_id,
        actor_id=CUSTOMER_ID,
        session_id="previous_session",
        messages=previous_interactions
    )
    print("✅ Customer history seeded successfully")
    print("⏳ Long-term memory processing will begin automatically...")

### タスク 1.2: メモリでエージェントを強化

大切な顧客が最近の注文に関する問題についてサポートチームに連絡します。彼らは好みを説明し、不満を共有し、エージェントと協力して問題を解決します。3週間後、関連する質問で再度連絡します。しかし今度は、すべてを繰り返さなければなりません - 好み、履歴、コンテキスト - エージェントは現在の会話セッションのみを記憶し、以前のセッションは記憶していないためです。これにより以下が発生します:
- **不満を持つ顧客** 情報を繰り返し繰り返さなければならない
- **非効率的なサポート** 以前のやり取りに基づいて構築できない
- **顧客満足度の低下** 非個人的で一般的な応答による

Amazon Bedrock AgentCore Memory は、AIエージェントが時間の経過とともにコンテキストを維持し、重要な事実を記憶し、一貫性のある個別化された体験を提供できるようにするマネージドサービスを提供することで、この制限に対処します。AgentCore Memory は2つのレベルで動作します:
- **短期メモリ**: 即座の会話コンテキストとセッションベースの情報(Strands Agent フレームワークによって自動的に処理)
- **長期メモリ**: 複数の会話から抽出された永続的な情報(事実、好み、要約を含む)(USER_PREFERENCE および SEMANTIC 戦略を使用した AgentCore Memory サービスを通じて実装)

プロトタイプを以下の例が可能な顧客認識アシスタントに変換します:
- **「お帰りなさい、Sarah さん!」** - 戻ってきた顧客を即座に認識
- **「先月のラップトップの問題についてフォローアップ」** - 関連する会話をシームレスに接続
- **「購入履歴に基づいて、これをお勧めします」** - 個別化された提案を提供

このタスクを完了すると、エージェントは統合されたメモリ機能を持つ以下のアーキテクチャを持つようになります:

<div style="text-align:left">
    <img src="images/architecture_lab2_memory.png" width="75%"/>
</div>

*画像の説明: 永続的な顧客コンテキストとパーソナライゼーションのための AgentCore Memory で強化されたエージェント*

**メモリ戦略の構成**: 2つのインテリジェントな戦略を組み合わせたメモリリソースを作成します:

| 戦略タイプ | 目的 | 顧客へのメリット |
|---------------|---------|------------------|
| USER_PREFERENCE | 顧客の好みと行動 | 「あなたが好むことを覚えています...」 |
| SEMANTIC | 事実情報とコンテキスト | 「以前の問題について...」 |

AgentCore Memory は、actorId を使用して長期メモリメッセージを論理的にグループ化するために名前空間を使用します:
- `support/customer/{actorId}/preferences`: ユーザー好み メモリ戦略用
- `support/customer/{actorId}/semantic`: セマンティック メモリ戦略用

In [ ]:
class CustomerSupportMemoryHooks(HookProvider):
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespaces = {
            i["type"]: i["namespaces"][0]
            for i in self.client.get_memory_strategies(self.memory_id)
        }

    def retrieve_customer_context(self, event: MessageAddedEvent):
        # エージェントが応答する前に顧客コンテキストを取得するために実行されるフック
        messages = event.agent.messages
        # ユーザーメッセージのみを処理(ツール結果は除く)
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                all_context = []
                # 各戦略の名前空間からメモリを取得、USER_PREFERENCE と SEMANTIC の両方
                for context_type, namespace in self.namespaces.items():
                    memories = self.client.retrieve_memories(
                        memory_id=self.memory_id,
                        namespace=namespace.format(actorId=self.actor_id),
                        query=user_query,
                        top_k=3,  # 関連性の高いメモリを上位3件取得
                    )
                    # メモリオブジェクトからテキストコンテンツを抽出
                    for memory in memories:
                        if isinstance(memory, dict):
                            content = memory.get("content", {})
                            if isinstance(content, dict):
                                text = content.get("text", "").strip()
                                if text:
                                    all_context.append(f"[{context_type.upper()}] {text}")
                
                # ユーザーメッセージに顧客コンテキストを前置
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0]["text"] = f"Customer Context:\n{context_text}\n\n{original_text}"
            except Exception as e:
                print(f"Failed to retrieve customer context: {e}")

    def save_support_interaction(self, event: AfterInvocationEvent):
        # エージェントが応答した後にやり取りをメモリに保存するために実行されるフック
        try:
            messages = event.agent.messages
            # ユーザーとアシスタントの両方のメッセージがある場合のみ保存
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                customer_query = None
                agent_response = None
                
                # 最新のユーザークエリとアシスタントの応答を検索
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not customer_query and "toolResult" not in msg["content"][0]:
                        customer_query = msg["content"][0]["text"]
                        break
                
                # やり取りを AgentCore Memory に保存
                if customer_query and agent_response:
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(customer_query, "USER"), (agent_response, "ASSISTANT")],
                    )
        except Exception as e:
            print(f"Failed to save support interaction: {e}")

    def register_hooks(self, registry: HookRegistry) -> None:
        # 両方のフックをエージェントのフックレジストリに登録
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)
        registry.add_callback(AfterInvocationEvent, self.save_support_interaction)

print("✅ Memory hooks defined - Automatic customer personalization enabled!")
print("🧠 Your agent will now remember customers and personalize every interaction")

メモリが配置されたメモリ強化エージェントを作成してテストし、顧客コンテキストを取得して応答をパーソナライズする方法を確認します:

In [ ]:
# フックを使用したメモリ強化エージェントを作成
memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

memory_agent = Agent(
    model=model,
    tools=[get_product_info, get_return_policy, get_technical_support, web_search],
    hooks=[memory_hooks],
    system_prompt=SYSTEM_PROMPT
)

print("✅ Memory-enhanced agent created!")
print("🧠 Agent will automatically retrieve customer context and save interactions")

In [ ]:
# メモリ処理の完了を待機
print("⏳ Waiting 30 seconds for memory processing to complete...")
time.sleep(30)

# メモリ呼び出しをテスト
print("🧠 Testing memory-enhanced agent...\n")
response = memory_agent("What are my laptop preferences?")
print("\n" + "="*50 + "\n")

以前にサポートチームとやり取りがあった「customer_001」という名前の戻ってきた顧客をシミュレートします。これは、AgentCore Memory が個々の会話を豊かで永続的な顧客インサイトに自動的に変換する方法を示しています。以前の顧客のやり取りを読み込み、AgentCore Memory がそれらを自動的に長期的な顧客インサイトに変換するのを見てください。

### AgentCore Gateway の作成

Gateway は、エージェントとバックエンド Lambda 関数の間の安全なプロキシとして機能します。AI エージェント専用に設計された API ゲートウェイと考えてください。

**作成するもの:**
- **Gateway インフラストラクチャ**: コア Gateway リソース
- **MCP プロトコル**: ツール通信の標準プロトコル
- **JWT 承認**: 先ほど作成した認証構成を使用
- **IAM ロール**: Lambda 関数を呼び出す権限

**次に起こること:**
1. Gateway を作成(このセル)
2. ツール定義を含む Lambda ターゲットを追加(次のセル)
   - 1つの Lambda 関数が複数のツールを処理: `check_warranty_status` と `web_search`
   - Gateway はツール名を Lambda に渡し、Lambda が適切なハンドラーにルーティング
3. エージェントを Gateway に接続

**注意**: CloudFormation テンプレートは、複数のツール操作を処理できる単一の Lambda 関数(`CustomerSupportLambda`)をデプロイしました。これは、各ツールに個別の Lambda 関数をデプロイするよりも効率的です。

In [ ]:
# SSM Parameter Store から認証構成を取得
# これらの値は CloudFormation テンプレートによって作成されました

# クライアント ID: リクエストを行っているアプリケーションを識別
machine_client_id = get_ssm_parameter("/app/customersupport/agentcore/machine_client_id")
print(f"Machine Client ID: {machine_client_id}")

# ディスカバリー URL: Gateway に Cognito の OAuth 構成の場所を伝える
# この URL はトークンエンドポイント、サポートされているスコープなどに関するメタデータを提供
cognito_discovery_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
print(f"Discovery URL: {cognito_discovery_url}")

# Gateway 用の JWT ベースの認証を構成
auth_config = {
    "customJWTAuthorizer": {
        # このクライアント ID からのトークンのみが受け入れられます
        "allowedClients": [machine_client_id],
        # Gateway はこの URL から OAuth メタデータを取得
        "discoveryUrl": cognito_discovery_url
    }
}

print("✅ Authentication configuration ready")

Strands Agents は、強く型付けされたイベントコールバックを通じてコンポーネントがエージェントの動作に反応または変更できるようにする強力なフックシステムを提供します。これにより、手動介入なしでメモリ操作が自動的に行われることが保証されます。

顧客がエージェントとやり取りするたびに、自動的に以下が行われます:
- **以前のやり取りと好みに基づいて会話をパーソナライズ**
- **新しいやり取りをメモリに追加** して、将来のパーソナライゼーションを継続的に改善

フック統合が行うこと:
- **応答前**: 関連する顧客コンテキストと好みを自動的に取得
- **応答後**: 新しいやり取りを AgentCore Memory に自動的に保存

顧客コンテキストの自動化をメモリフックで有効にします。

In [ ]:
class CreationFailedError(Exception):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)

gateway_client = boto3.client("bedrock-agentcore-control", region_name=REGION)
gateway_name = "customersupport-gw6"

try:
    print(f"Creating gateway: {gateway_name}")
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role"),
        protocolType="MCP",  # Model Context Protocol
        authorizerType="CUSTOM_JWT",  # 認証に JWT トークンを使用
        authorizerConfiguration=auth_config,  # 上記の認証構成
        description="Customer Support AgentCore Gateway",
    )
    gateway_id = create_response["gatewayId"]
    gateway_url = create_response["gatewayUrl"]
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)

    # Gateway の準備が整うまで待機
    print("Waiting for Gateway to be ready...")
    while True:
        status = gateway_client.get_gateway(gatewayIdentifier=gateway_id)['status']
        if status == 'READY':
            break
        elif status == 'FAILED':
            raise CreationFailedError("Gateway creation failed")
        else:
            print(f"  Status: {status}")
            time.sleep(5)

    print(f"✅ Gateway created successfully!")
    print(f"   Gateway ID: {gateway_id}")
    print(f"   Gateway URL: {gateway_url}")
    
except gateway_client.exceptions.ConflictException:
    # Gateway は既に存在するため、取得
    gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = gateway_response["gatewayUrl"]
    print(f"✅ Using existing gateway: {gateway_id}")
    
except CreationFailedError:
    print("\033[31m❌ Gateway creation failed. Check CloudWatch logs for details.\033[0m")

### Gateway にツールを追加

Lambda 関数を Gateway のターゲットとして登録し、エージェントが使用できる2つのツールを定義します:

**ツール 1: check_warranty_status**
- **目的**: 製品の保証ステータスを確認
- **パラメータ**: order_id(注文番号)
- **使用例**: 「注文 #12345 の保証ステータスを確認してください」

**ツール 2: web_search**
- **目的**: 最新情報をウェブで検索
- **パラメータ**: query(検索クエリ)
- **使用例**: 「最新のラップトップモデルを検索」

**重要な概念:**
- **単一の Lambda、複数のツール**: 1つの Lambda 関数が両方のツールを処理
- **ツールルーティング**: Gateway はツール名を Lambda に渡し、Lambda が適切なロジックを実行
- **MCP ツールスキーマ**: 各ツールには、エージェントが正しく使用する方法を理解するための詳細な説明とパラメータスキーマがあります

AgentCore Gateway は、呼び出すツールの名前を Lambda コンテキストに設定し、パラメータは Lambda イベントに渡されます。

In [ ]:
# Lambda ツールの API 仕様を読み込み
api_spec = [
    {
        "name": "check_warranty_status",
        "description": "シリアル番号とメールを使用して保証状況を確認する",
        "inputSchema": {
            "type": "object",
            "properties": {
                "serial_number": {"type": "string"},
                "customer_email": {"type": "string"}
            },
            "required": ["serial_number"]
        }
    },
    {
        "name": "web_search",
        "description": "最新情報をウェブで検索する",
        "inputSchema": {
            "type": "object",
            "properties": {
                "keywords": {"type": "string", "description": "検索クエリキーワード"},
                "region": {"type": "string", "description": "検索リージョン (e.g., us-en)"},
                "max_results": {"type": "integer", "description": "最大の結果"}
            },
            "required": ["keywords"]
        }
    }
]

# Gateway ターゲットを作成
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": get_ssm_parameter("/app/customersupport/agentcore/lambda_arn"),
            "toolSchema": {"inlinePayload": api_spec},
        }
    }
}

try:
    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaTarget",
        description="Lambda tools for customer support",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}],
    )
    print(f"✅ Gateway target created: {create_target_response['targetId']}")
except Exception as e:
    print(f"Gateway target may already exist: {str(e)}")

メモリが配置されたメモリ強化エージェントを作成してテストし、顧客コンテキストを取得して応答をパーソナライズする方法を確認します:

In [ ]:
def get_cognito_client_secret():
    # Cognito API を使用して Cognito クライアントシークレットを取得
    client = boto3.client("cognito-idp")
    response = client.describe_user_pool_client(
        UserPoolId=get_ssm_parameter("/app/customersupport/agentcore/userpool_id"),
        ClientId=get_ssm_parameter("/app/customersupport/agentcore/machine_client_id"),
    )
    return response["UserPoolClient"]["ClientSecret"]

def get_oauth_token():
    # クライアント認証情報フローを使用して Gateway 認証用の OAuth トークンを取得
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",  # OAuth 2.0 クライアント認証情報フロー
        "client_id": get_ssm_parameter("/app/customersupport/agentcore/machine_client_id"),
        "client_secret": get_cognito_client_secret(),
        "scope": get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_scope"),
    }
    # Cognito からアクセストークンをリクエスト
    response = requests.post(
        get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url"),
        headers=headers, data=data
    )
    return response.json()

# OAuth アクセストークン(JWT 形式)を取得
token_response = get_oauth_token()
access_token = token_response['access_token']

# Bearer トークン認証を使用した MCP クライアントを作成
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway_url,
        headers={"Authorization": f"Bearer {access_token}"},  # Authorization ヘッダーの JWT トークン
    )
)

print(f"✅ MCP client configured for gateway: {gateway_url}")

### エージェントを Gateway に接続

エージェントが Gateway を通じてツールを使用できるようにします。これには以下が必要です:

1. **認証**: Cognito からアクセストークンを取得
2. **MCP クライアント**: Gateway と通信するためのクライアントを作成
3. **エージェント構成**: Gateway ツールを使用するようにエージェントを更新

**認証フロー:**
```
1. Cognito トークンエンドポイントを取得
2. client_id と client_secret を使用してトークンをリクエスト
3. アクセストークンを受信
4. すべての Gateway リクエストでトークンを使用
```

すべてをまとめます - メモリフック + ローカルツール + Gateway ツール。これにより、ローカルツールと企業 Gateway ツールの両方をシームレスに使用できるハイブリッドアーキテクチャが作成されます。

In [ ]:
# 顧客コンテキスト用のメモリフックを初期化
memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

# Gateway への MCP クライアント接続を開始
mcp_client.start()
# Gateway から利用可能なツールを取得
gateway_tools = mcp_client.list_tools_sync()

# ローカルツールと一元化された Gateway ツールを組み合わせ
all_tools = [
    get_product_info,      # ローカルツール
    get_return_policy,     # ローカルツール
    get_technical_support, # ローカルツール
] + gateway_tools          # Gateway ツール - web_search、check_warranty_status

# メモリと Gateway 統合を備えた強化エージェントを作成
enhanced_agent = Agent(
    model=model,
    tools=all_tools,           # ローカル + Gateway ツール
    hooks=[memory_hooks],      # Automatic memory operations
    system_prompt=SYSTEM_PROMPT
)

print("✅ Enhanced Customer Support Agent created!")
print(f"📊 Total tools available: {len(all_tools)}")
print(f"🧠 Memory enabled with ID: {memory_id}")
print(f"🔒 Secure gateway integration: {gateway_url}")

### タスク 1.3: Gateway 統合と AgentCore Identity によるスケール

メモリが配置されたら、強力なツールに焦点を当て、その影響をスケールします。優れたエージェントには、独自およびサードパーティの API とデータを最大限に活用するツールが必要であり、これにより、これらのエージェントは内部および外部の顧客の両方のために物事を成し遂げることができます。ただし、エージェントツールの構築、セキュリティ保護、スケーリングは困難であり、顧客がエージェントのプロトタイプから本番環境での実際のエージェントビジネス価値への移行を妨げる重要なブロッカーになります。AgentCore Gateway は、AI エージェントが統一された **Model Context Protocol (MCP)** エンドポイントを通じて実世界のツールを発見、認証、呼び出しできるようにする接続レイヤーとして機能します。これは、数百の API、リソース、ツールを管理する企業にとって重要です。

主な利点:
- **完全管理された MCP サーバー** インフラストラクチャ管理なしのソリューション
- **既存の API の統合** と Lambda 関数
- **多様なツール間の統一インターフェース**
- **安全な認証** と承認
- **セマンティックツールの発見** と選択

##### 構築するもの:

**ツールの一元化と再利用性:**
- ウェブ検索をローカルツールから一元化された AgentCore Gateway に移行
- 既存の企業 Lambda 関数(保証チェック)を統合
- 複数のエージェントタイプがアクセスできる共有ツールインフラストラクチャを作成

**企業グレードのセキュリティ:**
- Cognito 統合による JWT ベースの認証を実装
- ゲートウェイアクセスのための安全なインバウンド承認を構成
- ツール使用のためのアイデンティティベースのアクセス制御を確立

これにより、ツールが一元管理され、複数のエージェントタイプで再利用可能なスケーラブルな基盤が作成され、コードの重複が排除され、メンテナンスが簡素化されます。

AgentCore Identity もこのプロセスに関与しています。これにより、AI エージェントは AWS リソースに安全にアクセスでき、Amazon Cognito と連携してインバウンド呼び出し元の認証を処理するのに役立ちます。また、AI エージェントがアウトバウンド認証を使用してサードパーティのツールとサービスに安全にアクセスできるようにしますが、この AgentCore Identity の機能はこのラボでは使用されません。

<div style="text-align:left">
    <img src="images/architecture_lab3_identity.png" width="75%"/>
</div>

このタスクを完了すると、エージェントは統合されたゲートウェイ機能を持つ以下のアーキテクチャを持つようになります:

<div style="text-align:left">
    <img src="images/architecture_lab3_gateway.png" width="75%"/>
</div>

*画像の説明: 安全で一元化されたツール管理と企業統合のための AgentCore Gateway で強化されたエージェント*

Lambda 関数を MCP 互換エンドポイントとして公開するために AgentCore Gateway を作成します。ツールの呼び出しを許可された呼び出し元を検証するために、OAuth 承認(MCP サーバーの標準)を使用して **インバウンド認証** を構成します。

In [ ]:
# Gateway ツールをテスト
print("🔍 Testing gateway web search...\n")
response2 = enhanced_agent("最新のiPhone 15のトラブルシューティングのヒントを検索して下さい。")
print("\n" + "="*50 + "\n")

In [ ]:
# 保証チェックをテスト
print("🛡️ Testing warranty check...\n")
response3 = enhanced_agent("シリアル番号ABC12345678の保証状況を確認してください。")
print("\n" + "="*50 + "\n")

In [ ]:
# 組み合わせた機能をテスト
print("🎯 Testing combined memory + gateway capabilities...\n")
response4 = enhanced_agent("ゲーミングヘッドフォンがまた必要になったので、最新のレビューも探して下さい。")
print("\n" + "="*50 + "\n")

### Gateway 認証の理解

AgentCore Gateway は **OAuth 2.0 と JWT トークン** を使用してツールへのアクセスを保護します。これにより、未承認のアプリケーションが Lambda 関数を呼び出すことを防ぎます。

**主要な概念:**

1. **認証プロバイダー**: Amazon Cognito がアイデンティティを管理し、トークンを発行
2. **クライアント認証情報**: エージェントは client_id と client_secret を使用(アプリケーションのユーザー名/パスワードのようなもの)
3. **JWT トークン**: エージェントが承認されていることを証明する短命のトークン
4. **許可されたクライアント**: Gateway にアクセスできるクライアント ID の許可リスト

**動作方法:**
```
エージェント → Cognito: 「これが私の client_id と client_secret です」
Cognito → エージェント: 「これがあなたの JWT アクセストークンです」
エージェント → Gateway: 「これが私のトークンです」
Gateway → Cognito: 「このトークンは有効で、許可されたクライアントからのものですか?」
Gateway → エージェント: 「アクセスが許可されました」
```

**セキュリティ注意**: これらの認証情報は事前に作成され、SSM Parameter Store に安全に保存されています。認証情報はコードにハードコードしないでください